In [1]:
# Module
import pandas as pd
import json
import urllib
import csv
import xml.etree.ElementTree as ET
import sys
import requests
import pprint
from os import name
import bs4
from lxml import html
from urllib.parse import urlencode, quote_plus, unquote
import re
import zipfile
import io
import os
import time

In [2]:
#OPENDART Batch Data
# 1.유가증권시장 전종목 리스트 뽑기
# 2. 전종목 재무제표 모으기.



# OPENDART Batch Data

## 1.유가증권시장 전종목 리스트 뽑기

In [3]:

# krx에서 csv 뽑고 종목 리스트 저장
# opendart에서 8자리 코드 뽑아오기
krx_kospi_list=pd.read_csv('data_2030_20230313.csv',encoding='cp949');
print(krx_kospi_list)
krx_kospi_list.to_csv('test.csv')

             표준코드    단축코드            한글 종목명  한글 종목약명  \
0    KR7095570008  095570         AJ네트웍스보통주   AJ네트웍스   
1    KR7006840003  006840          AK홀딩스보통주    AK홀딩스   
2    KR7282330000  282330         BGF리테일보통주   BGF리테일   
3    KR7027410000  027410            BGF보통주      BGF   
4    KR7138930003  138930        BNK금융지주보통주  BNK금융지주   
..            ...     ...               ...      ...   
938  KR7005010004  005010            휴스틸보통주      휴스틸   
939  KR7000541003  000545      흥국화재해상보험1우선주    흥국화재우   
940  KR7000542001  000547  흥국화재해상보험2우선주(신형)  흥국화재2우B   
941  KR7000540005  000540       흥국화재해상보험보통주     흥국화재   
942  KR7003280005  003280           흥아해운보통주     흥아해운   

                           영문 종목명         상장일   시장구분 증권구분  소속부   주식종류   액면가  \
0            AJ Networks Co.,Ltd.  2015/08/21  KOSPI   주권  NaN    보통주  1000   
1               AK Holdings, Inc.  1999/08/11  KOSPI   주권  NaN    보통주  5000   
2                      BGF Retail  2017/12/08  KOSPI   주권  NaN    보통주  1000   
3          

In [4]:
#krx에서 csv뽑고 종목 리스트 저장
krx_kospi_list=pd.read_csv('data_2030_20230313.csv',encoding='cp949');
krx_name_list=[]
temp=[]
#print(krx_kospi_list)

#krx_kospi_list 뽑기
for name in krx_kospi_list.iloc[:,2]:
    name=re.sub(r"[0-9]","",name)
    name=re.sub(r"보통주","",name)
    name=re.sub(r"우선주","",name)
    name=re.sub(r"\([^)]*\)","",name)
    temp.append(name)
for name in temp:
    if not name in krx_name_list:
        krx_name_list.append(name)
        #print(name)
#print(len(krx_name_list)) 825개 추출완료

#Corp Entity 비어있는 Dataframe 만들기
corp_df=pd.DataFrame(columns=("corp_id","name","common_code","preferred_code","opendart_code"))
id=0;
for name in krx_name_list:
    
    id+=1
    row={
        "corp_id":[id],
        "name":[name],
        "common_code":None,
        "preferred_code":None,
        "opendart_code":None
        }
    row=pd.DataFrame(row)
    corp_df=pd.concat([corp_df,row],ignore_index=True)
    
#print(corp_df)
for tuple in krx_kospi_list.itertuples():
    code=tuple[2]
    name=tuple[3]
    name=re.sub(r"\([^)]*\)","",name)
    if '보통주'in name:
        name=re.sub(r"[0-9]","",name)
        name=re.sub(r"보통주","",name)
        corp_df.loc[corp_df['name']==name,'common_code']=code
    elif '우선주'in name:
        name=re.sub(r"우선주","",name)
        if (str.isdigit(name)):
            if '1' in name:
                name=re.sub(r"[0-9]","",name)
                corp_df.loc[corp_df['name']==name,'preferred_code']=code
        else:
            name=re.sub(r"[0-9]","",name)
            corp_df.loc[corp_df['name']==name,'preferred_code']=code
    else:
        name=re.sub(r"[0-9]","",name)
        corp_df.loc[corp_df['name']==name,'common_code']=code
        
            
    #print(row[3])
print(corp_df)
corp_df.to_csv("CORP_Entity.csv",index=False)

    corp_id       name common_code preferred_code opendart_code
0         1     AJ네트웍스      095570           None          None
1         2      AK홀딩스      006840           None          None
2         3     BGF리테일      282330           None          None
3         4        BGF      027410           None          None
4         5    BNK금융지주      138930           None          None
..      ...        ...         ...            ...           ...
820     821  휴니드테크놀러지스      005870           None          None
821     822        휴비스      079980           None          None
822     823        휴스틸      005010           None          None
823     824   흥국화재해상보험      000540         000547          None
824     825       흥아해운      003280           None          None

[825 rows x 5 columns]


In [5]:
#opendart에서 8자리 종목뽑아오기
#opendart_key이용
opendart_key='13adcf79a1486a73eb1d950890dcc93f3af4273b'
# url로 zipfile받기
url="https://opendart.fss.or.kr/api/corpCode.xml"
params={'crtfc_key':opendart_key};
response_corpCode=requests.get(url,params=params)
zip_corpCode=zipfile.ZipFile(io.BytesIO(response_corpCode.content))
zip_corpCode.extractall(os.getcwd()+'\\download')

In [6]:
#opendart에서 8자리 종목뽑아오기
#corp_df활성화 되어 있어야함

#CORPCODE.xml 뽑아오기
tree=ET.parse('CORPCODE.xml')
root=tree.getroot()
#ET.dump(tree)

for element in root:
    if(element[2].text!=" "):
        name=element[2].text
        code=element[0].text
        corp_df.loc[corp_df["common_code"]==name,"opendart_code"]=code
corp_df.to_csv("CORP_Entity.csv",index=False)

## 2. 전종목 재무제표 모으기

In [17]:
# 2. 전종목 재무제표 모으기.
opendart_code_list=[]
for code in corp_df.iloc[:,4]:
    #print(code)
    opendart_code_list.append(code)

#불러오고 안불러오고 확인 list
none_list=[]
success_list=[]

#단일회사 전체 재무제표 url\
opendart_key='13adcf79a1486a73eb1d950890dcc93f3af4273b'
url="https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json?"
#~61까지 됨
for corp_code in corp_df.iloc[818:826,4]:
    for year in range(2015,2024):
        
        bsns_year=year
        # CFS : 연결재무제표 (consolidated)
        for reprt_code in range(11011,11015):
            time.sleep(0.3)
            params={'crtfc_key':opendart_key,'corp_code':corp_code,'bsns_year':bsns_year,'reprt_code':reprt_code,'fs_div':"CFS"}
            response=requests.get(url,params=params)
            temp_json=response.json()
            file_path="./corpfs/"+corp_code+"/"+corp_code+"_CFS_"+str(bsns_year)+"_"+str(reprt_code)+".json"
            if(temp_json['status']=="000"):
                with open(file_path,'w') as jsonfile:
                    json.dump(temp_json,jsonfile)
            else:
                none_list.append(params)
            #OFS : 재무제표
            params={'crtfc_key':opendart_key,'corp_code':corp_code,'bsns_year':bsns_year,'reprt_code':reprt_code,'fs_div':"OFS"}
            response=requests.get(url,params=params)
            temp_json=response.json()
            file_path="./corpfs/"+corp_code+"/"+corp_code+"_OFS_"+str(bsns_year)+"_"+str(reprt_code)+".json"
            if(temp_json['status']=="000"):
                with open(file_path,'w') as jsonfile:
                    json.dump(temp_json,jsonfile)
            else:
                none_list.append(params)
            
        # OFS : 재무제표
#         for reprt_code in range[11011,11014]:
#             params={'crtfc_key':opendart_key,'corp_code':corp_code,'bsns_year':bsns_year,'reprt_code':11011,'fs_div':"OFS"}
#             response=requests.get(url,params=params)
        
# for code in corp_df.iloc[200:400,4]:
#     print(code)
# for code in corp_df.iloc[400:600,4]:
#     print(code)
# for code in corp_df.iloc[600:400,4]:
#     print(code)
    


In [76]:
corp_code="00365387"
bsns_year="2015"
params={'crtfc_key':opendart_key,'corp_code':corp_code,'bsns_year':bsns_year,'reprt_code':11011,'fs_div':"OFS"}
response=requests.get(url,params=params)
temp_json=response.json()
print(temp_json)

{'status': '000', 'message': '정상', 'list': [{'rcept_no': '20160524000353', 'reprt_code': '11011', 'bsns_year': '2015', 'corp_code': '00365387', 'sj_div': 'BS', 'sj_nm': '재무상태표', 'account_id': 'ifrs_CurrentAssets', 'account_nm': '유동자산', 'account_detail': '-', 'thstrm_nm': '제 16 기', 'thstrm_amount': '91870888881', 'frmtrm_nm': '제 15 기', 'frmtrm_amount': '37214915600', 'bfefrmtrm_nm': '제 15 기초', 'bfefrmtrm_amount': '31882201971', 'ord': '1', 'currency': 'KRW'}, {'rcept_no': '20160524000353', 'reprt_code': '11011', 'bsns_year': '2015', 'corp_code': '00365387', 'sj_div': 'BS', 'sj_nm': '재무상태표', 'account_id': 'ifrs_CashAndCashEquivalents', 'account_nm': '현금및현금성자산', 'account_detail': '-', 'thstrm_nm': '제 16 기', 'thstrm_amount': '36459117395', 'frmtrm_nm': '제 15 기', 'frmtrm_amount': '7043224361', 'bfefrmtrm_nm': '제 15 기초', 'bfefrmtrm_amount': '1225111059', 'ord': '2', 'currency': 'KRW'}, {'rcept_no': '20160524000353', 'reprt_code': '11011', 'bsns_year': '2015', 'corp_code': '00365387', 'sj_div

In [80]:
print(none_list)

[{'crtfc_key': '13adcf79a1486a73eb1d950890dcc93f3af4273b', 'corp_code': '00125080', 'bsns_year': 2015, 'reprt_code': 11012, 'fs_div': 'CFS'}, {'crtfc_key': '13adcf79a1486a73eb1d950890dcc93f3af4273b', 'corp_code': '00125080', 'bsns_year': 2015, 'reprt_code': 11012, 'fs_div': 'OFS'}, {'crtfc_key': '13adcf79a1486a73eb1d950890dcc93f3af4273b', 'corp_code': '00125080', 'bsns_year': 2015, 'reprt_code': 11013, 'fs_div': 'CFS'}, {'crtfc_key': '13adcf79a1486a73eb1d950890dcc93f3af4273b', 'corp_code': '00125080', 'bsns_year': 2015, 'reprt_code': 11013, 'fs_div': 'OFS'}, {'crtfc_key': '13adcf79a1486a73eb1d950890dcc93f3af4273b', 'corp_code': '00125080', 'bsns_year': 2015, 'reprt_code': 11014, 'fs_div': 'CFS'}, {'crtfc_key': '13adcf79a1486a73eb1d950890dcc93f3af4273b', 'corp_code': '00125080', 'bsns_year': 2015, 'reprt_code': 11014, 'fs_div': 'OFS'}, {'crtfc_key': '13adcf79a1486a73eb1d950890dcc93f3af4273b', 'corp_code': '00125080', 'bsns_year': 2022, 'reprt_code': 11011, 'fs_div': 'CFS'}, {'crtfc_key'

In [16]:
a=0
for i in corp_df.iloc[:,4]:
    a+=1
    print(i)
    if(i=="00595191"):
        print(a)

print(a)

00365387
00125080
01263022
00219097
00858364
00122579
00148540
00129013
00149026
00163345
00115694
00159102
00160843
00878915
00118345
00109693
00128971
01524093
00110608
00151298
00165583
01437186
01568413
00126487
00500254
00120030
00164636
01310269
00633835
00161116
01042775
00299002
00154426
00980122
00225706
00149947
00632304
00688996
01622720
00103042
00115676
00101220
00193009
00159740
00191472
00587466
00112378
00593032
00120021
00105873
00356370
01515323
00231363
00105961
00401731
00356361
00503668
00105855
00104698
00105952
00525934
00120076
00121534
00266961
00983271
01391033
00158316
00116301
00108038
00148896
00138279
00125150
00152039
00137809
00136457
00138260
00815369
00181943
00181712
00139889
00144164
00131780
00508344
01535150
01596425
01386916
00631518
00131850
01267170
00159023
00398792
00648721
00134477
00225159
00125530
00138297
00480455
00406046
00409672
00117300
00142865
00104768
00100939
00255619
00129554
00101433
00101488
01208885
00143527
00101628
00101664
0

In [63]:
# for corp_code in corp_df.iloc[:,4]:
#     folder_path="./corpfs/"+corp_code
#     try: 
#         if not os.path.exists(folder_path):
#             os.makedirs(folder_path)
#     except OSError:
#         print("Error: Creating driectory")